Use the trainer_env conda evn  
set the path for the transformer library to the env

`//home/dosisiddhesh/transformers/src/transformers`

In [1]:
!pwd

/home/dosisiddhesh/latex_model/custom_trainer


In [2]:
import os 
import sys
# add the 'home/dosisiddhesh/transformers' path to the system path
sys.path.append('/home/dosisiddhesh/transformers/src')
print(sys.path)

['/home/dosisiddhesh/latex_model/custom_trainer', '/home/dosisiddhesh/miniconda3/envs/trainer_env/lib/python312.zip', '/home/dosisiddhesh/miniconda3/envs/trainer_env/lib/python3.12', '/home/dosisiddhesh/miniconda3/envs/trainer_env/lib/python3.12/lib-dynload', '', '/home/dosisiddhesh/miniconda3/envs/trainer_env/lib/python3.12/site-packages', '/home/dosisiddhesh/transformers/src']


In [10]:
from transformers import trainer

ImportError: cannot import name 'GenerationMixin' from 'transformers.generation' (/home/dosisiddhesh/miniconda3/envs/trainer_env/lib/python3.12/site-packages/transformers/generation/__init__.py)

In [9]:
!pip install huggingface_hub -q
!pip install transformers -q
!pip install torch 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.9 MB/s eta 0:00:0000:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.7 MB/s eta 0:00:0000:0100:02m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 1.4 MB/s eta 0:00:00:00:01


# notebook version of run_clm_no_trainer.py to debug the loss

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

#### code section 1

In [2]:
#!/usr/bin/env python
# coding=utf-8
# Copyright 2021 The HuggingFace Inc. team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
Fine-tuning the library models for causal language modeling (GPT, GPT-2, CTRL, ...)
on a text file or a dataset without using HuggingFace Trainer.

Here is the full list of checkpoints on the hub that can be fine-tuned by this script:
https://huggingface.co/models?filter=text-generation
"""
# You can also adapt this script on your own causal language modeling task. Pointers for this are left as comments.

''' To Run this file, use the following command:
------------------------------------------------
Original
________________________________________________
CUDA_VISIBLE_DEVICES=2 python run_clm_no_trainer.py \
    --dataset_name wikitext \
    --dataset_config_name wikitext-2-raw-v1 \
    --model_name_or_path openai-community/gpt2 \
    --output_dir /tmp/test-clm

    

----------------------------------------------
Modified
______________________________________________
CUDA_VISIBLE_DEVICES=2 python run_clm_no_trainer.py \
    --dataset_name wikitext \
    --dataset_config_name wikitext-2-raw-v1 \
    --model_name_or_path custom_mistral \
    --output_dir /tmp/test-clm \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --block_size 128 \
    --float16

------------------------------------------------
With Gpt model architecture
______________________________________________
python run_clm_no_trainer.py \
    
------------------------------------------------
'''

import argparse
import json
import logging
import math
import os
import random
from itertools import chain
from pathlib import Path

import datasets
import torch
from accelerate import Accelerator, DistributedType
from accelerate.logging import get_logger
from accelerate.utils import set_seed
from datasets import load_dataset, DatasetDict
from huggingface_hub import HfApi
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

import transformers
from transformers import (
    CONFIG_MAPPING,
    MODEL_MAPPING,
    AutoConfig,
    AutoModelForCausalLM,
    MistralForCausalLM,
    MistralConfig,
    AutoTokenizer,
    SchedulerType,
    default_data_collator,
    get_scheduler,
)
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version

from pynvml import *
def gpu_usage():
    def print_gpu_utilization():
        nvmlInit()
        handle = nvmlDeviceGetHandleByIndex(0)
        info = nvmlDeviceGetMemoryInfo(handle)
        print(f"GPU memory occupied from nvmlInit: {info.used//1024**2} MB.")
        logger.info(f"GPU memory occupied from nvmlInit: {info.used//1024**2} MB.")

    print("+----------------------------------------------------------------------------------+")
    a,b = torch.cuda.mem_get_info()
    gpu_mem_usage = (b-a)/(2**20)
    print(f"GPU memory usage before cleaning cache: {gpu_mem_usage:.2f} MB")
    logger.info(f"GPU memory usage before cleaning cache: {gpu_mem_usage:.2f} MB")

    torch.cuda.empty_cache()
    
    a,b = torch.cuda.mem_get_info()
    gpu_mem_usage = (b-a)/(2**20)
    print(f"GPU memory usage after cleaning cache: {gpu_mem_usage:.2f} MB")
    logger.info(f"GPU memory usage after cleaning cache: {gpu_mem_usage:.2f} MB")
    print_gpu_utilization()
    print("+----------------------------------------------------------------------------------+")

class Parameter:
    def __init__(self, name, value, use_cache=True):
        self.name = name
        self.D_emb,self.Vocal,self.d_head,self.d_FF,self.N_Layer,self.N_Head,self.KV_Head,self.Window = value
        self.use_cache = use_cache

from prettytable import PrettyTable
def model_size_and_parameters(model, logger = None):
    table = PrettyTable(["Modules", "Parameters"])
    model_size = sum(t.numel() for t in model.parameters())
    total_params = 0
    one_layer_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params += params
        if "layers.0" in name:
            one_layer_params += params
    print(table)
    print(f"MISTRAL model size: {model_size/1000**2:.1f}M parameters")
    print(f"Total Trainable Params: {total_params/10**6:.4f}M")
    print(f"Total Trainable Params in one layer: {one_layer_params/10**6:.4f}M")
    print("Original Model type:",model.dtype)

    if logger:
        logger.info(table)
        logger.info(f"MISTRAL model size: {model_size/1000**2:.1f}M parameters")
        logger.info(f"Total Trainable Params: {total_params/10**6:.4f}M")
        logger.info(f"Total Trainable Params in one layer: {one_layer_params/10**6:.4f}M")
        logger.info(f"Original Model type:{model.dtype}")
        
    return total_params, one_layer_params



# Will error if the minimal version of Transformers is not installed. Remove at your own risks.
check_min_version("4.40.0.dev0")

logger = get_logger(__name__)

require_version("datasets>=1.8.0", "To fix: pip install -r examples/pytorch/language-modeling/requirements.txt")

MODEL_CONFIG_CLASSES = list(MODEL_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)



/home/dosisiddhesh/miniconda3/envs/trainer_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
'''
    --dataset_name wikitext \
    --dataset_config_name wikitext-2-raw-v1 \
    --model_name_or_path custom_mistral \
    --output_dir /tmp/test-clm \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --block_size 128 \
    --float16
'''

def parse_args():
    parser = argparse.ArgumentParser(description="Finetune a transformers model on a causal language modeling task")
    parser.add_argument(
        "--dataset_name",
        type=str,
        default='wikitext',
        help="The name of the dataset to use (via the datasets library).",
    )
    # add argument whether to load the model in float 16
    parser.add_argument(
        "--float16",
        action="store_false",
        help="Whether or not to use float16 (mixed precision).",
    )        
    parser.add_argument(
        "--dataset_config_name",
        type=str,
        default='wikitext-2-raw-v1',
        help="The configuration name of the dataset to use (via the datasets library).",
    )
    parser.add_argument(
        "--train_file", type=str, default=None, help="A csv, txt or a json file containing the training data."
    )
    parser.add_argument(
        "--validation_file", type=str, default=None, help="A csv, txt or a json file containing the validation data."
    )
    parser.add_argument(
        "--validation_split_percentage",
        default=5,
        help="The percentage of the train set used as validation set in case there's no validation split",
    )
    parser.add_argument(
        "--model_name_or_path",
        default='custom_mistral',
        # default='openai-community/gpt2',
        # default='facebook/galactica-125m',
        type=str,
        help="Path to pretrained model or model identifier from huggingface.co/models.",
        required=False,
    )
    parser.add_argument(
        "--config_name",
        type=str,
        default=None,
        help="Pretrained config name or path if not the same as model_name",
    )
    parser.add_argument(
        "--tokenizer_name",
        type=str,
        default=None,
        help="Pretrained tokenizer name or path if not the same as model_name",
    )
    parser.add_argument(
        "--use_slow_tokenizer",
        action="store_true",
        help="If passed, will use a slow tokenizer (not backed by the 🤗 Tokenizers library).",
    )
    parser.add_argument(
        "--per_device_train_batch_size",
        type=int,
        default=1,
        help="Batch size (per device) for the training dataloader.",
    )
    parser.add_argument(
        "--per_device_eval_batch_size",
        type=int,
        default=1,
        help="Batch size (per device) for the evaluation dataloader.",
    )
    parser.add_argument(
        "--learning_rate",
        type=float,
        default=5e-5,
        help="Initial learning rate (after the potential warmup period) to use.",
    )
    parser.add_argument("--weight_decay", type=float, default=0.0, help="Weight decay to use.")
    parser.add_argument("--num_train_epochs", type=int, default=1, help="Total number of training epochs to perform.")
    parser.add_argument(
        "--max_train_steps",
        type=int,
        default=None,
        help="Total number of training steps to perform. If provided, overrides num_train_epochs.",
    )
    parser.add_argument(
        "--gradient_accumulation_steps",
        type=int,
        default=1,
        help="Number of updates steps to accumulate before performing a backward/update pass.",
    )
    parser.add_argument(
        "--lr_scheduler_type",
        type=SchedulerType,
        default="linear",
        help="The scheduler type to use.",
        choices=["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"],
    )
    parser.add_argument(
        "--num_warmup_steps", type=int, default=0, help="Number of steps for the warmup in the lr scheduler."
    )
    parser.add_argument("--output_dir", type=str, default='./test-clm', help="Where to store the final model.")
    parser.add_argument("--seed", type=int, default=None, help="A seed for reproducible training.")
    parser.add_argument(
        "--model_type",
        type=str,
        default=None,
        help="Model type to use if training from scratch.",
        choices=MODEL_TYPES,
    )
    parser.add_argument(
        "--block_size",
        type=int,
        default=128,
        help=(
            "Optional input sequence length after tokenization. The training dataset will be truncated in block of"
            " this size for training. Default to the model max input length for single sentence inputs (take into"
            " account special tokens)."
        ),
    )
    parser.add_argument(
        "--preprocessing_num_workers",
        type=int,
        default=None,
        help="The number of processes to use for the preprocessing.",
    )
    parser.add_argument(
        "--overwrite_cache", action="store_true", help="Overwrite the cached training and evaluation sets"
    )
    parser.add_argument(
        "--no_keep_linebreaks", action="store_true", help="Do not keep line breaks when using TXT files."
    )
    parser.add_argument("--push_to_hub", action="store_true", help="Whether or not to push the model to the Hub.")
    parser.add_argument(
        "--hub_model_id", type=str, help="The name of the repository to keep in sync with the local `output_dir`."
    )
    parser.add_argument("--hub_token", type=str, help="The token to use to push to the Model Hub.")
    parser.add_argument(
        "--trust_remote_code",
        type=bool,
        default=False,
        help=(
            "Whether or not to allow for custom models defined on the Hub in their own modeling files. This option "
            "should only be set to `True` for repositories you trust and in which you have read the code, as it will "
            "execute code present on the Hub on your local machine."
        ),
    )
    parser.add_argument(
        "--checkpointing_steps",
        type=str,
        default=None,
        help="Whether the various states should be saved at the end of every n steps, or 'epoch' for each epoch.",
    )
    parser.add_argument(
        "--resume_from_checkpoint",
        type=str,
        default=None,
        help="If the training should continue from a checkpoint folder.",
    )
    parser.add_argument(
        "--with_tracking",
        action="store_true",
        help="Whether to enable experiment trackers for logging.",
    )
    parser.add_argument(
        "--report_to",
        type=str,
        default="all",
        help=(
            'The integration to report the results and logs to. Supported platforms are `"tensorboard"`,'
            ' `"wandb"`, `"comet_ml"` and `"clearml"`. Use `"all"` (default) to report to all integrations. '
            "Only applicable when `--with_tracking` is passed."
        ),
    )
    parser.add_argument(
        "--low_cpu_mem_usage",
        action="store_true",
        help=(
            "It is an option to create the model as an empty shell, then only materialize its parameters when the pretrained weights are loaded. "
            "If passed, LLM loading time and RAM consumption will be benefited."
        ),
    )
    args = parser.parse_args('')

    # Sanity checks
    if args.dataset_name is None and args.train_file is None and args.validation_file is None:
        raise ValueError("Need either a dataset name or a training/validation file.")
    else:
        if args.train_file is not None:
            extension = args.train_file.split(".")[-1]
            if extension not in ["csv", "json", "txt"]:
                raise ValueError("`train_file` should be a csv, json or txt file.")
        if args.validation_file is not None:
            extension = args.validation_file.split(".")[-1]
            if extension not in ["csv", "json", "txt"]:
                raise ValueError("`validation_file` should be a csv, json or txt file.")

    if args.push_to_hub:
        if args.output_dir is None:
            raise ValueError("Need an `output_dir` to create a repo when `--push_to_hub` is passed.")

    return args



In [4]:
args = parse_args()

args

Namespace(dataset_name='wikitext', float16=True, dataset_config_name='wikitext-2-raw-v1', train_file=None, validation_file=None, validation_split_percentage=5, model_name_or_path='custom_mistral', config_name=None, tokenizer_name=None, use_slow_tokenizer=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, learning_rate=5e-05, weight_decay=0.0, num_train_epochs=1, max_train_steps=None, gradient_accumulation_steps=1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, num_warmup_steps=0, output_dir='./test-clm', seed=None, model_type=None, block_size=128, preprocessing_num_workers=None, overwrite_cache=False, no_keep_linebreaks=False, push_to_hub=False, hub_model_id=None, hub_token=None, trust_remote_code=False, checkpointing_steps=None, resume_from_checkpoint=None, with_tracking=False, report_to='all', low_cpu_mem_usage=False)

In [5]:
# Sending telemetry. Tracking the example usage helps us better allocate resources to maintain them. The
# information sent is the one passed as arguments along with your Python/PyTorch versions.
send_example_telemetry("run_clm_no_trainer", args)

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
# If we're using tracking, we also need to initialize it here and it will by default pick up all supported trackers
# in the environment
accelerator_log_kwargs = {}

if args.with_tracking:
    accelerator_log_kwargs["log_with"] = args.report_to
    accelerator_log_kwargs["project_dir"] = args.output_dir

accelerator = Accelerator(gradient_accumulation_steps=args.gradient_accumulation_steps, **accelerator_log_kwargs)

# Make one log on every process with the configuration for debugging.
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state, main_process_only=False)
if accelerator.is_local_main_process:
    datasets.utils.logging.set_verbosity_warning()
    transformers.utils.logging.set_verbosity_info()
else:
    datasets.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_error()

# If passed along, set the training seed now.
if args.seed is not None:
    set_seed(args.seed)

# Handle the repository creation
if accelerator.is_main_process:
    if args.push_to_hub:
        # Retrieve of infer repo_name
        repo_name = args.hub_model_id
        if repo_name is None:
            repo_name = Path(args.output_dir).absolute().name
        # Create repo and retrieve repo_id
        api = HfApi()
        repo_id = api.create_repo(repo_name, exist_ok=True, token=args.hub_token).repo_id

        with open(os.path.join(args.output_dir, ".gitignore"), "w+") as gitignore:
            if "step_*" not in gitignore:
                gitignore.write("step_*\n")
            if "epoch_*" not in gitignore:
                gitignore.write("epoch_*\n")
    elif args.output_dir is not None:
        os.makedirs(args.output_dir, exist_ok=True)
accelerator.wait_for_everyone()

# Get the datasets: you can either provide your own CSV/JSON/TXT training and evaluation files (see below)
# or just provide the name of one of the public datasets available on the hub at https://huggingface.co/datasets/
# (the dataset will be downloaded automatically from the datasets Hub).
#
# For CSV/JSON files, this script will use the column called 'text' or the first column if no column called
# 'text' is found. You can easily tweak this behavior (see below).
#
# In distributed training, the load_dataset function guarantee that only one local process can concurrently
# download the dataset.
if args.dataset_name is not None:
    # Downloading and loading a dataset from the hub.
    # raw_datasets = load_dataset(args.dataset_name, args.dataset_config_name)
    # Load 10% of each split using slicing
    train_dataset = load_dataset(args.dataset_name, args.dataset_config_name, split='train[:1%]')
    test_dataset = load_dataset(args.dataset_name, args.dataset_config_name, split='test[:10%]')
    validation_dataset = load_dataset(args.dataset_name, args.dataset_config_name, split='validation[:10%]')

    # Combine them into a DatasetDict
    raw_datasets = DatasetDict({
        'train': train_dataset,
        'test': test_dataset,
        'validation': validation_dataset
    })
    print("Raw datasets: ", raw_datasets)
    print("---------------------------------------------------")

    if "validation" not in raw_datasets.keys():
        raw_datasets["validation"] = load_dataset(
            args.dataset_name,
            args.dataset_config_name,
            split=f"train[:{args.validation_split_percentage}%]",
        )
        raw_datasets["train"] = load_dataset(
            args.dataset_name,
            args.dataset_config_name,
            split=f"train[{args.validation_split_percentage}%:]",
        )
else:
    data_files = {}
    dataset_args = {}
    if args.train_file is not None:
        data_files["train"] = args.train_file
        extension = args.train_file.split(".")[-1]
    if args.validation_file is not None:
        data_files["validation"] = args.validation_file
        extension = args.validation_file.split(".")[-1]
    if extension == "txt":
        extension = "text"
        dataset_args["keep_linebreaks"] = not args.no_keep_linebreaks
    raw_datasets = load_dataset(extension, data_files=data_files, **dataset_args)
    # If no validation data is there, validation_split_percentage will be used to divide the dataset.
    if "validation" not in raw_datasets.keys():
        raw_datasets["validation"] = load_dataset(
            extension,
            data_files=data_files,
            split=f"train[:{args.validation_split_percentage}%]",
            **dataset_args,
        )
        raw_datasets["train"] = load_dataset(
            extension,
            data_files=data_files,
            split=f"train[{args.validation_split_percentage}%:]",
            **dataset_args,
        )

# See more about loading any type of standard or custom dataset (from files, python dict, pandas DataFrame, etc) at
# https://huggingface.co/docs/datasets/loading_datasets.

# Load pretrained model and tokenizer
#
# In distributed training, the .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.
        
##################################################################################################
if args.config_name:
    config = AutoConfig.from_pretrained(
        args.config_name,
        trust_remote_code=args.trust_remote_code,
    )
elif args.model_name_or_path == "custom_mistral":
    print("--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+")
    print("Loading local custom config")
    D_emb = 128 #4*1024
    Vocal = 30000
    d_head = 128# + 64
    d_FF = 128 # 14336
    N_Layer = 1
    N_Head = 32
    KV_Head = 8
    Window = 4096 #8192
    # data_row = 100
    value = [D_emb,Vocal,d_head,d_FF,N_Layer,N_Head,KV_Head,Window]
    param = Parameter('MISTRAL', value)
    config = MistralConfig(
            vocab_size=param.Vocal,
            hidden_size=param.D_emb,
            intermediate_size=param.d_FF,
            num_hidden_layers=param.N_Layer,
            num_attention_heads=param.N_Head,
            num_key_value_heads=param.KV_Head,
            hidden_act="relu",#"silu",
            max_position_embeddings=4096 * 32,
            initializer_range=0.02,
            rms_norm_eps=1e-6,
            use_cache=param.use_cache,
            # use_reentrant=False,
            pad_token_id=None,
            bos_token_id=1,
            eos_token_id=2,
            tie_word_embeddings=False,
            rope_theta=10000.0,
            sliding_window=param.Window,
            attention_dropout=0.0,
            torch_dtype=torch.float16,
        )
    print("CONFIG: ", config)
    

elif args.model_name_or_path:
    config = AutoConfig.from_pretrained(
        args.model_name_or_path,
        trust_remote_code=args.trust_remote_code,
    )
else:
    config = CONFIG_MAPPING[args.model_type]()
    logger.warning("You are instantiating a new config instance from scratch.")
        
if args.tokenizer_name:
    tokenizer = AutoTokenizer.from_pretrained(
        args.tokenizer_name, use_fast=not args.use_slow_tokenizer, trust_remote_code=args.trust_remote_code
    )
elif args.model_name_or_path == "custom_mistral":
    print("--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+")
    print("Loading local custom tokenizer")
    TOKENIZER_HF_ST_PATH = '/home/dosisiddhesh/MISTRAL_EXP/model/hf_tokenizer_1.0%_30000_new'
    tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_HF_ST_PATH)
    print("Len of tokenizer before adding special tokens", len(tokenizer))
    tokenizer.add_special_tokens({'pad_token': '<pad>',
                                        'cls_token': '<cls>',
                                        'sep_token': '<sep>',
                                        'mask_token': '<mask>',
                                        'unk_token': '<unk>',
                                        'bos_token': '<bos>',
                                        'eos_token': '<eos>'
                                    })
    # print the vocab of the tokenizer
    # print("Vocab of the tokenizer: ", self.tokenizer.get_vocab())
    print("Len of tokenizer after adding special tokens", len(tokenizer))

elif args.model_name_or_path:
    tokenizer = AutoTokenizer.from_pretrained(
        args.model_name_or_path, use_fast=not args.use_slow_tokenizer, trust_remote_code=args.trust_remote_code
    )
else:
    raise ValueError(
        "You are instantiating a new tokenizer from scratch. This is not supported by this script. "
        "You can do it from another script, save it, and load it from here, using --tokenizer_name."
    )



if args.model_name_or_path == "custom_mistral":
    print("--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+")
    print("Loading local custom model")
    model = MistralForCausalLM(config)


elif args.model_name_or_path:
    model = AutoModelForCausalLM.from_pretrained(
        args.model_name_or_path,
        from_tf=bool(".ckpt" in args.model_name_or_path),
        config=config,
        low_cpu_mem_usage=args.low_cpu_mem_usage,
        trust_remote_code=args.trust_remote_code,
    )


else:
    logger.info("Training new model from scratch")
    model = AutoModelForCausalLM.from_config(config, trust_remote_code=args.trust_remote_code)


if args.float16:
    print(f'Model loaded in {model.dtype} with config')
    model = model.half()
    print(f'Model reduced to half precision in {model.dtype}')

model_size_and_parameters(model, logger)
model.to(0)
##################################################################################################

# We resize the embeddings only when necessary to avoid index errors. If you are creating a model from scratch
# on a small vocab and want a smaller embedding size, remove this test.
embedding_size = model.get_input_embeddings().weight.shape[0]
if len(tokenizer) > embedding_size:
    model.resize_token_embeddings(len(tokenizer))


04/01/2024 15:22:27 - INFO - __main__ - Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2
}



Raw datasets:  DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 367
    })
    test: Dataset({
        features: ['text'],
        num_rows: 436
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 376
    })
})
---------------------------------------------------
--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+
Loading local custom config
CONFIG:  MistralConfig {
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "relu",
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 128,
  "max_position_embeddings": 131072,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 1,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-06,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.40.0.dev0",
  "use_cache": true,
  "vocab_size": 30000
}

--+--+--+--+--+--+

04/01/2024 15:23:05 - INFO - __main__ - +------------------------------------------------+------------+
|                    Modules                     | Parameters |
+------------------------------------------------+------------+
|           model.embed_tokens.weight            |  3840000   |
|     model.layers.0.self_attn.q_proj.weight     |   16384    |
|     model.layers.0.self_attn.k_proj.weight     |    4096    |
|     model.layers.0.self_attn.v_proj.weight     |    4096    |
|     model.layers.0.self_attn.o_proj.weight     |   16384    |
|      model.layers.0.mlp.gate_proj.weight       |   16384    |
|       model.layers.0.mlp.up_proj.weight        |   16384    |
|      model.layers.0.mlp.down_proj.weight       |   16384    |
|     model.layers.0.input_layernorm.weight      |    128     |
| model.layers.0.post_attention_layernorm.weight |    128     |
|               model.norm.weight                |    128     |
|                 lm_head.weight                 |  3840000   |


Model loaded in torch.float32 with config
Model reduced to half precision in torch.float16
+------------------------------------------------+------------+
|                    Modules                     | Parameters |
+------------------------------------------------+------------+
|           model.embed_tokens.weight            |  3840000   |
|     model.layers.0.self_attn.q_proj.weight     |   16384    |
|     model.layers.0.self_attn.k_proj.weight     |    4096    |
|     model.layers.0.self_attn.v_proj.weight     |    4096    |
|     model.layers.0.self_attn.o_proj.weight     |   16384    |
|      model.layers.0.mlp.gate_proj.weight       |   16384    |
|       model.layers.0.mlp.up_proj.weight        |   16384    |
|      model.layers.0.mlp.down_proj.weight       |   16384    |
|     model.layers.0.input_layernorm.weight      |    128     |
| model.layers.0.post_attention_layernorm.weight |    128     |
|               model.norm.weight                |    128     |
|            

### check the model types and precision 

In [6]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(30000, 128)
    (layers): ModuleList(
      (0): MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=128, out_features=128, bias=False)
          (k_proj): Linear(in_features=128, out_features=32, bias=False)
          (v_proj): Linear(in_features=128, out_features=32, bias=False)
          (o_proj): Linear(in_features=128, out_features=128, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=128, out_features=128, bias=False)
          (up_proj): Linear(in_features=128, out_features=128, bias=False)
          (down_proj): Linear(in_features=128, out_features=128, bias=False)
          (act_fn): ReLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  )
  (lm_head): Linear(in

In [7]:
config.to_dict()

{'vocab_size': 30000,
 'max_position_embeddings': 131072,
 'hidden_size': 128,
 'intermediate_size': 128,
 'num_hidden_layers': 1,
 'num_attention_heads': 32,
 'sliding_window': 4096,
 'num_key_value_heads': 8,
 'hidden_act': 'relu',
 'initializer_range': 0.02,
 'rms_norm_eps': 1e-06,
 'use_cache': True,
 'rope_theta': 10000.0,
 'attention_dropout': 0.0,
 'return_dict': True,
 'output_hidden_states': False,
 'output_attentions': False,
 'torchscript': False,
 'torch_dtype': 'float16',
 'use_bfloat16': False,
 'tf_legacy_loss': False,
 'pruned_heads': {},
 'tie_word_embeddings': False,
 'chunk_size_feed_forward': 0,
 'is_encoder_decoder': False,
 'is_decoder': False,
 'cross_attention_hidden_size': None,
 'add_cross_attention': False,
 'tie_encoder_decoder': False,
 'max_length': 20,
 'min_length': 0,
 'do_sample': False,
 'early_stopping': False,
 'num_beams': 1,
 'num_beam_groups': 1,
 'diversity_penalty': 0.0,
 'temperature': 1.0,
 'top_k': 50,
 'top_p': 1.0,
 'typical_p': 1.0,
 'rep

In [8]:
raw_datasets['train']['text'][1]

' = Valkyria Chronicles III = \n'

In [9]:


# Preprocessing the datasets.
# First we tokenize all the texts.
column_names = raw_datasets["train"].column_names
text_column_name = "text" if "text" in column_names else column_names[0]

def tokenize_function(examples):
    return tokenizer(examples[text_column_name], return_token_type_ids=False)

with accelerator.main_process_first():
    tokenized_datasets = raw_datasets.map(
        tokenize_function,
        batched=True,
        num_proc=args.preprocessing_num_workers,
        remove_columns=column_names,
        load_from_cache_file=not args.overwrite_cache,
        desc="Running tokenizer on dataset",
    )




In [10]:
type(tokenized_datasets['train']['input_ids'][1][0])

int

In [11]:

if args.block_size is None:
    block_size = tokenizer.model_max_length
    if block_size > config.max_position_embeddings:
        logger.warning(
            f"The tokenizer picked seems to have a very large `model_max_length` ({tokenizer.model_max_length}). "
            f"Using block_size={min(1024, config.max_position_embeddings)} instead. You can change that default value by passing --block_size xxx."
        )
        block_size = min(1024, config.max_position_embeddings)
else:
    if args.block_size > tokenizer.model_max_length:
        logger.warning(
            f"The block_size passed ({args.block_size}) is larger than the maximum length for the model "
            f"({tokenizer.model_max_length}). Using block_size={tokenizer.model_max_length}."
        )
    block_size = min(args.block_size, tokenizer.model_max_length)
    print("-------------------- the block size is set to: ", block_size)


# Main data processing function that will concatenate all texts from our dataset and generate chunks of block_size.
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, and if the total_length < block_size  we exclude this batch and return an empty dict.
    # We could add padding if the model supported it instead of this drop, you can customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

# Note that with `batched=True`, this map processes 1,000 texts together, so group_texts throws away a remainder
# for each of those groups of 1,000 texts. You can adjust that batch_size here but a higher value might be slower
# to preprocess.
#
# To speed up this part, we use multiprocessing. See the documentation of the map method for more information:
# https://huggingface.co/docs/datasets/process#map

with accelerator.main_process_first():
    lm_datasets = tokenized_datasets.map(
        group_texts,
        batched=True,
        num_proc=args.preprocessing_num_workers,
        load_from_cache_file=not args.overwrite_cache,
        desc=f"Grouping texts in chunks of {block_size}",
    )


# def convert_to_fp16(batch):
#     # Convert tensors that are not used as indices for embeddings to torch.float16
#     # Assuming 'input_ids', 'attention_mask', etc. are the keys for tensors that should be converted
#     float_keys = ['input_ids']  # Add or remove keys as per your data structure
#     return {k: torch.tensor(v, dtype=torch.float16) if k in float_keys else torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}

# # Your mapping function remains the same
# lm_datasets = lm_datasets.map(
#     convert_to_fp16,
#     batched=True,
#     num_proc=args.preprocessing_num_workers,
#     load_from_cache_file=not args.overwrite_cache,
#     desc="Converting appropriate tensors to float16"
# )

train_dataset = lm_datasets["train"]
eval_dataset = lm_datasets["validation"]


# Log a few random samples from the training set:
for index in random.sample(range(len(train_dataset)), 3):
    logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

04/01/2024 15:23:06 - INFO - __main__ - Sample 90 of the training set: {'input_ids': [2862, 12412, 1054, 1890, 47, 1430, 2633, 5778, 74, 3955, 1012, 11141, 1025, 14808, 2730, 1026, 497, 74, 9359, 1054, 16516, 1046, 47, 35, 9008, 1525, 1191, 7921, 1015, 14808, 1025, 1351, 5749, 1061, 1901, 1061, 72, 7624, 1168, 85, 1514, 122, 45, 1015, 13042, 1596, 40, 9755, 13071, 1347, 10141, 45, 4090, 6103, 1054, 1015, 19132, 13058, 13574, 122, 1008, 1158, 1603, 47, 1111, 2285, 1178, 2302, 1191, 7921, 1233, 6633, 45, 1027, 1308, 98, 2065, 1451, 1777, 1313, 10577, 1038, 2452, 45, 76, 1558, 4877, 5683, 24938, 2379, 1855, 2862, 1015, 2065, 25451, 5157, 47, 26668, 7169, 1027, 8611, 1015, 71, 2558, 13574, 122, 1052, 1847, 2177, 45, 1027, 1858, 1591, 1009, 1015, 2490, 1596, 1027, 9407, 1025, 1015, 2298, 19920, 1039, 1032, 7322, 47, 1111], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

-------------------- the block size is set to:  128


In [12]:




def custom_collate_fn(batch):
    # Use the default data collator to process the batch
    batch = default_data_collator(batch)
    
    # Convert all tensors in the batch to torch.float16
    # batch = {key: value.to(torch.float16) if isinstance(value, torch.Tensor) else value for key, value in batch.items()}
    
    return batch



# DataLoaders creation:
train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=custom_collate_fn, batch_size=args.per_device_train_batch_size
)
eval_dataloader = DataLoader(
    eval_dataset, collate_fn=custom_collate_fn, batch_size=args.per_device_eval_batch_size
)

for batch in train_dataloader:
    print("Batch keys: ", batch.keys())
    # print dtype of all tensors in the batch
    for key, value in batch.items():
        print(f"{key}: {value.dtype}")

    
    break


Batch keys:  dict_keys(['input_ids', 'attention_mask', 'labels'])
input_ids: torch.int64
attention_mask: torch.int64
labels: torch.int64


In [13]:

# Optimizer
# Split weights in two groups, one with weight decay and the other not.
no_decay = ["bias", "layer_norm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=args.learning_rate)

# Scheduler and math around the number of training steps.
overrode_max_train_steps = False
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)
if args.max_train_steps is None:
    args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
    overrode_max_train_steps = True

lr_scheduler = get_scheduler(
    name=args.lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=args.num_warmup_steps * accelerator.num_processes,
    num_training_steps=args.max_train_steps
    if overrode_max_train_steps
    else args.max_train_steps * accelerator.num_processes,
)

# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader, lr_scheduler
)


for batch in train_dataloader:
    print("Batch keys: ", batch.keys())
    # print dtype of all tensors in the batch
    for key, value in batch.items():
        print(f"{key}: {value.dtype}")
    
    break


Batch keys:  dict_keys(['input_ids', 'attention_mask', 'labels'])
input_ids: torch.int64
attention_mask: torch.int64
labels: torch.int64


In [14]:

# On TPU, the tie weights in our model have been disconnected, so we need to restore the ties.
if accelerator.distributed_type == DistributedType.TPU:
    model.tie_weights()

# We need to recalculate our total training steps as the size of the training dataloader may have changed.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)
if overrode_max_train_steps:
    args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
# Afterwards we recalculate our number of training epochs
args.num_train_epochs = math.ceil(args.max_train_steps / num_update_steps_per_epoch)

# Figure out how many steps we should save the Accelerator states
checkpointing_steps = args.checkpointing_steps
if checkpointing_steps is not None and checkpointing_steps.isdigit():
    checkpointing_steps = int(checkpointing_steps)

# We need to initialize the trackers we use, and also store our configuration.
# The trackers initializes automatically on the main process.
if args.with_tracking:
    experiment_config = vars(args)
    # TensorBoard cannot log Enums, need the raw value
    experiment_config["lr_scheduler_type"] = experiment_config["lr_scheduler_type"].value
    accelerator.init_trackers("clm_no_trainer", experiment_config)

# Train!
total_batch_size = args.per_device_train_batch_size * accelerator.num_processes * args.gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num Epochs = {args.num_train_epochs}")
logger.info(f"  Instantaneous batch size per device = {args.per_device_train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
logger.info(f"  Total optimization steps = {args.max_train_steps}")
# Only show the progress bar once on each machine.
progress_bar = tqdm(range(args.max_train_steps), disable=not accelerator.is_local_main_process)
completed_steps = 0
starting_epoch = 0

# Potentially load in the weights and states from a previous save
if args.resume_from_checkpoint:
    if args.resume_from_checkpoint is not None or args.resume_from_checkpoint != "":
        checkpoint_path = args.resume_from_checkpoint
        path = os.path.basename(args.resume_from_checkpoint)
    else:
        # Get the most recent checkpoint
        dirs = [f.name for f in os.scandir(os.getcwd()) if f.is_dir()]
        dirs.sort(key=os.path.getctime)
        path = dirs[-1]  # Sorts folders by date modified, most recent checkpoint is the last
        checkpoint_path = path
        path = os.path.basename(checkpoint_path)

    accelerator.print(f"Resumed from checkpoint: {checkpoint_path}")
    accelerator.load_state(checkpoint_path)
    # Extract `epoch_{i}` or `step_{i}`
    training_difference = os.path.splitext(path)[0]

    if "epoch" in training_difference:
        starting_epoch = int(training_difference.replace("epoch_", "")) + 1
        resume_step = None
        completed_steps = starting_epoch * num_update_steps_per_epoch
    else:
        # need to multiply `gradient_accumulation_steps` to reflect real steps
        resume_step = int(training_difference.replace("step_", "")) * args.gradient_accumulation_steps
        starting_epoch = resume_step // len(train_dataloader)
        completed_steps = resume_step // args.gradient_accumulation_steps
        resume_step -= starting_epoch * len(train_dataloader)

# update the progress_bar if load from checkpoint
progress_bar.update(completed_steps)


/home/dosisiddhesh/miniconda3/envs/trainer_env/lib/python3.12/site-packages/accelerate/utils/dataclasses.py:301: FutureWarning: The `TPU` of `<enum 'DistributedType'>` is deprecated and will be removed in v1.0.0. Please use the `XLA` instead.
  warnings.warn(
04/01/2024 15:23:07 - INFO - __main__ - ***** Running training *****
04/01/2024 15:23:07 - INFO - __main__ -   Num examples = 165
04/01/2024 15:23:07 - INFO - __main__ -   Num Epochs = 1
04/01/2024 15:23:07 - INFO - __main__ -   Instantaneous batch size per device = 1
04/01/2024 15:23:07 - INFO - __main__ -   Total train batch size (w. parallel, distributed & accumulation) = 1
04/01/2024 15:23:07 - INFO - __main__ -   Gradient Accumulation steps = 1
04/01/2024 15:23:07 - INFO - __main__ -   Total optimization steps = 165
  0%|          | 0/165 [00:00<?, ?it/s]

#### debug section

In [15]:

print(f"{'-'*48} + {'-'*13} + {'-'*28} + {'-'*30}")
print(f"{'name':49}| {'dtype':14}| {'shape':29}| {'requires_grad':30}")
print(f"{'-'*48} + {'-'*13} + {'-'*28} + {'-'*30}")
for name, param in model.named_parameters():
    print(f"{str(name):50} {str(param.dtype):15} {str(param.shape):30} {('requires_grad' if param.requires_grad else 'non grad'):30}")

------------------------------------------------ + ------------- + ---------------------------- + ------------------------------
name                                             | dtype         | shape                        | requires_grad                 
------------------------------------------------ + ------------- + ---------------------------- + ------------------------------
model.embed_tokens.weight                          torch.float16   torch.Size([30000, 128])       requires_grad                 
model.layers.0.self_attn.q_proj.weight             torch.float16   torch.Size([128, 128])         requires_grad                 
model.layers.0.self_attn.k_proj.weight             torch.float16   torch.Size([32, 128])          requires_grad                 
model.layers.0.self_attn.v_proj.weight             torch.float16   torch.Size([32, 128])          requires_grad                 
model.layers.0.self_attn.o_proj.weight             torch.float16   torch.Size([128, 128])        

In [16]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Gradient of {name}: {param.grad}")

Gradient of model.embed_tokens.weight: None
Gradient of model.layers.0.self_attn.q_proj.weight: None
Gradient of model.layers.0.self_attn.k_proj.weight: None
Gradient of model.layers.0.self_attn.v_proj.weight: None
Gradient of model.layers.0.self_attn.o_proj.weight: None
Gradient of model.layers.0.mlp.gate_proj.weight: None
Gradient of model.layers.0.mlp.up_proj.weight: None
Gradient of model.layers.0.mlp.down_proj.weight: None
Gradient of model.layers.0.input_layernorm.weight: None
Gradient of model.layers.0.post_attention_layernorm.weight: None
Gradient of model.norm.weight: None
Gradient of lm_head.weight: None


In [17]:
def detect_nan(tensor, name="Tensor"):

    print(f"{name:50} Gradient isnan: {torch.isnan(tensor).any()}")
    print(f"{name:50} Gradient isinf: {torch.isinf(tensor).any()}")

In [18]:
gpu_usage()
from torch.cuda import amp
epoch = 1

model.train()
if args.with_tracking:
    total_loss = 0
if args.resume_from_checkpoint and epoch == starting_epoch and resume_step is not None:
    # We skip the first `n` batches in the dataloader when resuming from a checkpoint
    active_dataloader = accelerator.skip_first_batches(train_dataloader, resume_step)
else:
    active_dataloader = train_dataloader

losses = []
for step, batch in enumerate(active_dataloader):
    with accelerator.accumulate(model):
        # outputs = model(input_ids=input_ids.to(torch.float16))
        # batch = {k: v.to(torch.float16) for k, v in batch.items()}
        with amp.autocast():
            outputs = model(**batch)
        logits = outputs.logits.to(torch.float16)
        print("=======================================================================")
        print(f'logits type: {logits.dtype}')
        print("-----------------------------------------------------------------------")

        print(f'Loss: {outputs.loss}')
        print("-----------------------------------------------------------------------")
        print(f'logits: {outputs.logits}')
        print("-----------------------------------------------------------------------")
        print(f'past_key_values: {outputs.past_key_values}')
        print("-----------------------------------------------------------------------")
        print(f'logits shape: {outputs.logits.shape}')  
        print("=======================================================================")
        print(f'past_key_values length: {len(outputs.past_key_values)}')
        for j in range(len(outputs.past_key_values)):
            print(f'{" "*5}-----------------------------------------------------------------------')
            print(f'{" "*5}past_key_values[{j}] length (layers): {len(outputs.past_key_values[j])}')
            for i in range(len(outputs.past_key_values[j])):
                print(f'{" "*10}past_key_values[{j}][{i}].shape: {outputs.past_key_values[j][i].shape}')

        # check if any nan of inf values in the logits or past_key_values using torch.isnan() and torch.isinf()
        if torch.isnan(outputs.logits).any() or torch.isinf(outputs.logits).any():
            print("Found nan or inf values in logits")
        else:
            print("No nan or inf values in logits")
        

        detect_nan(outputs.logits, "Logits")

        # print the float type of the model
        print(f"Model type: {model.dtype}")
        # print the float type of the outputs
        print(f"Outputs type: {outputs.loss.dtype}")
        print(f'Outputs logits {outputs.logits.dtype}')
        print(f'Outputs past_key_values {outputs.past_key_values[0][0].dtype}')
        print(f'Outputs loss {outputs.loss.dtype}')

        print("##########################################################################################################")
        for name, param in model.named_parameters():
            if param.requires_grad:
                print(f"Gradient of {name}: {param.grad}")
                if param.grad is not None:
                    detect_nan(param.grad, name)
        print("==========================================================================================================")
        for name, param in model.named_parameters():
            if param.requires_grad:
                print(f"Data of {name}: {param.data}")
                if param.data is not None:
                    detect_nan(param.data, name)
        print("==========================================================================================================")
        input("Enter to continue: ...")

        loss = outputs.loss
        loss.to(torch.float16)
        losses.append(accelerator.gather_for_metrics(loss.repeat(args.per_device_train_batch_size)))

        # We keep track of the loss at each epoch
        if args.with_tracking:
            total_loss += loss.detach().float()

        # computes the gradients of the loss with respect to the model parameters
        accelerator.backward(loss) #  The accelerator.backward method is used instead of the usual loss.backward() to potentially handle distributed training specifics.
        # loss.backward() #  The accelerator.backward method is used instead of the usual loss.backward() to potentially handle distributed training specifics.
        print("##########################################################################################################")
        for name, param in model.named_parameters():
            if param.requires_grad:
                print(f"Gradient of {name}: {param.grad}")
                if param.grad is not None:
                    detect_nan(param.grad, name)
        print("==========================================================================================================")
        for name, param in model.named_parameters():
            if param.requires_grad:
                print(f"Data of {name}: {param.data}")
                if param.data is not None:
                    detect_nan(param.data, name)
        print("==========================================================================================================")
        input("Enter to continue: ...")

        #_______________________________________________________________________________________________________________
        # -------------------------------------------------------------- adding grading clipping -----------------------
        max_grad_norm = 0.5

        grad_norm = accelerator.clip_grad_norm_(
            model.parameters(),
            max_grad_norm,
        )

        print(f"Gradient norm: {grad_norm}")
        
        print("##########################################################################################################")
        for name, param in model.named_parameters():
            if param.requires_grad:
                print(f"Gradient of {name}: {param.grad}")
                if param.grad is not None:
                    detect_nan(param.grad, name)
        print("==========================================================================================================")
        for name, param in model.named_parameters():
            if param.requires_grad:
                print(f"Data of {name}: {param.data}")
                if param.data is not None:
                    detect_nan(param.data, name)
        print("==========================================================================================================")
        input("Enter to continue: ...")

        #_______________________________________________________________________________________________________________


        # updates the model parameters based on the computed gradients.
        optimizer.step()
        print("##########################################################################################################")
        for name, param in model.named_parameters():
            if param.requires_grad:
                print(f"Gradient of {name}: {param.grad}")
                if param.grad is not None:
                    detect_nan(param.grad, name)
        print("==========================================================================================================")
        for name, param in model.named_parameters():
            if param.requires_grad:
                print(f"Data of {name}: {param.data}")
                if param.data is not None:
                    detect_nan(param.data, name)
        print("==========================================================================================================")
        input("Enter to continue: ...")

        # updates the learning rate according to the scheduler's policy
        lr_scheduler.step()
        # clears the old gradients to prevent them from accumulating with the gradients of the next batch.
        optimizer.zero_grad()
        print("##########################################################################################################")
        for name, param in model.named_parameters():
            if param.requires_grad:
                print(f"Gradient of {name}: {param.grad}")
                if param.grad is not None:
                    detect_nan(param.grad, name)
        print("==========================================================================================================")
        for name, param in model.named_parameters():
            if param.requires_grad:
                print(f"Data of {name}: {param.data}")
                if param.data is not None:
                    detect_nan(param.data, name)
        print("==========================================================================================================")
        input("Enter to continue: ...")
        break
    # Checks if the accelerator has performed an optimization step behind the scenes
    if accelerator.sync_gradients:
        progress_bar.update(1)
        completed_steps += 1

    if isinstance(checkpointing_steps, int):
        if completed_steps % checkpointing_steps == 0:
            output_dir = f"step_{completed_steps}"
            if args.output_dir is not None:
                output_dir = os.path.join(args.output_dir, output_dir)
            accelerator.save_state(output_dir)
    if completed_steps >= args.max_train_steps:
        break


04/01/2024 15:23:08 - INFO - __main__ - GPU memory usage before cleaning cache: 3270.31 MB
04/01/2024 15:23:08 - INFO - __main__ - GPU memory usage after cleaning cache: 3270.31 MB
04/01/2024 15:23:08 - INFO - __main__ - GPU memory occupied from nvmlInit: 32503 MB.


+----------------------------------------------------------------------------------+
GPU memory usage before cleaning cache: 3270.31 MB
GPU memory usage after cleaning cache: 3270.31 MB
GPU memory occupied from nvmlInit: 32503 MB.
+----------------------------------------------------------------------------------+
logits type: torch.float16
-----------------------------------------------------------------------
Loss: 10.345694541931152
-----------------------------------------------------------------------
logits: tensor([[[-0.0424,  0.1405, -0.3186,  ..., -0.0536, -0.0767,  0.0967],
         [-0.2224, -0.1342,  0.3086,  ..., -0.0615,  0.0247,  0.1748],
         [-0.1973, -0.1361,  0.3901,  ...,  0.0219,  0.0798,  0.1747],
         ...,
         [ 0.0336, -0.1544,  0.4487,  ..., -0.1309, -0.0045,  0.1150],
         [ 0.2319,  0.3330,  0.2759,  ...,  0.0732,  0.0803,  0.2098],
         [ 0.1400, -0.0151, -0.1193,  ...,  0.4390, -0.2406, -0.0137]]],
       device='cuda:0', grad_fn=<ToCop

In [ ]:

losses = torch.cat(losses)
try:
    train_loss = torch.mean(losses)
    perplexity = math.exp(train_loss)
except OverflowError:
    perplexity = float("inf")
logger.info(f"epoch {epoch}: perplexity: {perplexity} train_loss: {train_loss}")


model.eval()
losses = []
for step, batch in enumerate(eval_dataloader):
    with torch.no_grad():
        outputs = model(**batch)

    loss = outputs.loss
    losses.append(accelerator.gather_for_metrics(loss.repeat(args.per_device_eval_batch_size)))

losses = torch.cat(losses)
try:
    eval_loss = torch.mean(losses)
    perplexity = math.exp(eval_loss)
except OverflowError:
    perplexity = float("inf")

logger.info(f"epoch {epoch}: perplexity: {perplexity} eval_loss: {eval_loss}")

if args.with_tracking:
    accelerator.log(
        {
            "perplexity": perplexity,
            "eval_loss": eval_loss,
            "train_loss": total_loss.item() / len(train_dataloader),
            "epoch": epoch,
            "step": completed_steps,
        },
        step=completed_steps,
    )

if args.push_to_hub and epoch < args.num_train_epochs - 1:
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(
        args.output_dir, is_main_process=accelerator.is_main_process, save_function=accelerator.save
    )
    if accelerator.is_main_process:
        tokenizer.save_pretrained(args.output_dir)
        api.upload_folder(
            commit_message=f"Training in progress epoch {epoch}",
            folder_path=args.output_dir,
            repo_id=repo_id,
            repo_type="model",
            token=args.hub_token,
        )

if args.checkpointing_steps == "epoch":
    output_dir = f"epoch_{epoch}"
    if args.output_dir is not None:
        output_dir = os.path.join(args.output_dir, output_dir)
    accelerator.save_state(output_dir)


if args.with_tracking:
    accelerator.end_training()

if args.output_dir is not None:
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(
        args.output_dir, is_main_process=accelerator.is_main_process, save_function=accelerator.save
    )
    if accelerator.is_main_process:
        tokenizer.save_pretrained(args.output_dir)
        if args.push_to_hub:
            api.upload_folder(
                commit_message="End of training",
                folder_path=args.output_dir,
                repo_id=repo_id,
                repo_type="model",
                token=args.hub_token,
            )
        with open(os.path.join(args.output_dir, "all_results.json"), "w") as f:
            json.dump({"perplexity": perplexity}, f)
